# SAM Encoder for Image Embedding

In [1]:
cd /home/icb/hanyi.zhang/Detection_Head/segment-anything

/ictstr01/home/icb/hanyi.zhang/Detection_Head/segment-anything


In [2]:
from segment_anything import sam_model_registry
from segment_anything.separate_sam_encoder import SamEncoder

sam_checkpoint = "/home/icb/hanyi.zhang/Detection_Head/segment-anything/sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = "cuda"
#device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

sam_image_encoder = SamEncoder(sam, device)

In [4]:
import os
from PIL import Image
import numpy as np

def emb_images(image_folder, embed_folder, encoder):
    """
    Process images to obtain embeddings and save them.

    Parameters:
    - image_folder: str, path to the folder containing source images
    - embed_folder: str, path to the folder where embeddings will be saved
    - encoder: the encoder object that processes images to obtain embeddings
    """
    # Create the embedding directory if it doesn't exist
    os.makedirs(embed_folder, exist_ok=True)
    
    # List all files in the image directory
    images = os.listdir(image_folder)
    
    # Iterate over each image file
    for i, img in enumerate(images, start=1):
        if i % 100 == 0:
            print(f"Processed {i} images.")
        
        # Construct full path to the image file
        img_path = os.path.join(image_folder, img)
        #print(img_path)
        
        # Load the image file and convert it to a numpy array
        with Image.open(img_path) as image:
            image = image.convert('RGB')
            img_np = np.array(image)
        
        # Process the image to get its embedding
        img_embed = encoder.set_image(img_np)
        
        # Replace the original extension with 'npy' for the output file
        embed_path = os.path.join(embed_folder, os.path.splitext(img)[0] + '.npy')
        
        # Save the embedding to the specified path
        np.save(embed_path, img_embed.cpu().detach().numpy())


In [5]:
# Training dataset
base_folder = '/home/icb/hanyi.zhang/own_organoid_dataset'
img_patch_dir = os.path.join(base_folder, 'image_patches_filter')
embed_folder = os.path.join(base_folder, 'images_emb')
emb_images(img_patch_dir, embed_folder, sam_image_encoder)

Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.


# Use SAM built-in function for mask coordinates processing

In [6]:
import numpy as np
import os

def preprocess_boxes(mask_dir, box_dir):
    # Create the directory
    os.makedirs(box_dir, exist_ok=True)

    processed_count = 0
    # Loop through each file in the directory
    for file_name in os.listdir(mask_dir):
        label_file = np.load(os.path.join(mask_dir, file_name), allow_pickle=True)
        transformed_coord = sam_image_encoder.set_box_coordinates(label_file)

        # Normalization
        transformed_coord = transformed_coord / 1024

        # save boxes
        np.save(os.path.join(box_dir, file_name), transformed_coord)

        # Increment the counter for each augmented image
        processed_count += 1
        
        # Print the number of processed images every 100 images
        if processed_count % 100 == 0:
            print(f"Processed {processed_count} images.")


In [7]:
# Training dataset
base_folder = '/home/icb/hanyi.zhang/own_organoid_dataset'
mask_dir = os.path.join(base_folder, 'label_patches_filter')
box_dir = os.path.join(base_folder, 'bboxes')
preprocess_boxes(mask_dir, box_dir)

Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.


# Transform coordinates format

In [ ]:
import numpy as np
import os
import pandas as pd

def trans_boxes(ori_box_file, trans_box_dir, sam_image_encoder):
    # Create the directory if it doesn't exist
    os.makedirs(trans_box_dir, exist_ok=True)

    # Load the CSV file into a DataFrame
    df = pd.read_csv(ori_box_file)
    
    # Group by image path to process each image separately
    grouped = df.groupby(df.columns[0])
    
    # Initialize a counter for processed images
    processed_count = 0
    
    for image_path, group in grouped:
        # Extract the file name from the image path
        file_name = os.path.basename(image_path)
        
        # Extract the original size from the file name
        if '450x450' in file_name:
            ori_size = (450, 450)
        elif '300x300' in file_name:
            ori_size = (300, 300)
        else:
            # Default or additional logic if needed
            raise ValueError(f"Unknown size in file name: {file_name}")
        
        # Extract the bounding box coordinates as a numpy array
        bboxes = group.iloc[:, 1:5].values
        
        # Transform the box coordinates using SAM's image encoder
        transformed_coord = sam_image_encoder.trans_box_coordinates(bboxes, ori_size)
        
        # Normalize the transformed coordinates
        transformed_coord = transformed_coord / 1024
        
        # Replace file extension with .npy
        npy_file_name = os.path.splitext(file_name)[0] + '.npy'
        
        # Save the transformed coordinates as a .npy file
        np.save(os.path.join(trans_box_dir, npy_file_name), transformed_coord)
        
        # Increment the counter
        processed_count += 1
        
        # Print the number of processed images every 100 images
        if processed_count % 100 == 0:
            print(f"Processed {processed_count} images.")
            


In [ ]:
label_file = "/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/train_labels.csv"
df = pd.read_csv(label_file)
grouped = df.groupby(df.columns[0])

In [ ]:
i=0
names_labels = []
for image_path, group in grouped:
    i+=1
    file_name = os.path.basename(image_path)
    names_labels.append(file_name)

names_images = os.listdir('/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/training/images')

In [ ]:
len(names_images)

In [ ]:
len(names_labels)

In [ ]:
# Find all files in names_images but not in names_labels
missing_labels = [image for image in names_images if image not in names_labels]

# Save the result in a new list
print(f"Files in names_images but not in names_labels: {len(missing_labels)}")
print(missing_labels)

In [ ]:
import os

# Assuming missing_labels is your list of missing files

# Directory where the .npy files are located
npy_dir = '/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/training/images_emb'

# Replace 'jpg' with 'npy' in the missing_labels and delete the corresponding files
for name in missing_labels:
    npy_name = name.replace('.jpg', '.npy')
    npy_path = os.path.join(npy_dir, npy_name)
    
    if os.path.exists(npy_path):
        os.remove(npy_path)
        print(f"Deleted: {npy_path}")
    else:
        print(f"File not found: {npy_path}")


In [ ]:
label_file = "/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/train_labels.csv"
output_folder = "/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/training/boxes"
trans_boxes(label_file, output_folder, sam_image_encoder)

In [ ]:
label_file = "/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/test_labels.csv"
output_folder = "/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/test/boxes"
trans_boxes(label_file, output_folder, sam_image_encoder)

# Transform bboxes to the original size

In [ ]:
import numpy as np
from PIL import Image
from copy import deepcopy
from typing import Tuple
import random

# Define the transformation functions
def get_preprocess_shape(oldh: int, oldw: int, long_side_length: int) -> Tuple[int, int]:
    """
    Compute the output size given input size and target long side length.
    """
    scale = long_side_length * 1.0 / max(oldh, oldw)
    newh, neww = oldh * scale, oldw * scale
    neww = int(neww + 0.5)
    newh = int(newh + 0.5)
    return (newh, neww)

def inverse_coords(coords: np.ndarray, original_size: Tuple[int, int], target_length=1024) -> np.ndarray:
    """
    Inverse transformation of coordinates from resized back to original.
    """
    old_h, old_w = original_size
    new_h, new_w = get_preprocess_shape(old_h, old_w, target_length)
    coords = deepcopy(coords).astype(float)
    coords[..., 0] = coords[..., 0] * (old_w / new_w)
    coords[..., 1] = coords[..., 1] * (old_h / new_h)
    return coords

def inverse_boxes(boxes: np.ndarray, original_size: Tuple[int, int], target_length=1024) -> np.ndarray:
    """
    Inverse transformation of boxes from resized back to original.
    """
    boxes = inverse_coords(boxes.reshape(-1, 2, 2), original_size, target_length)
    return boxes.reshape(-1, 4)

def convert_boxes(boxes):
    # Convert the boxes from center format to [y_min, x_min, y_max, x_max] format
    converted_boxes = np.zeros_like(boxes)
    converted_boxes[:, 0] = boxes[:, 1] - boxes[:, 3] / 2.0  # x_min
    converted_boxes[:, 1] = boxes[:, 0] - boxes[:, 2] / 2.0  # y_min
    converted_boxes[:, 2] = boxes[:, 1] + boxes[:, 3] / 2.0  # x_max
    converted_boxes[:, 3] = boxes[:, 0] + boxes[:, 2] / 2.0  # y_max
    return converted_boxes

img_folder = '/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/training/images'
box_folder = '/home/icb/hanyi.zhang/public_organoid_datasets/OrgaQuant_DeepOrga_dataset/Intestinal_Organoid_Dataset/training/boxes'

img_name = random.choice(os.listdir(img_folder))
img = Image.open(os.path.join(img_folder, img_name))
box = np.load(os.path.join(box_folder, img_name.replace('jpg', 'npy')))

img_np = np.array(img)

original_size = img.size  # (width, height)
original_size = (original_size[1], original_size[0])  # Convert to (height, width)

original_boxes = inverse_boxes(convert_boxes(box) * 1024, original_size)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Convert the image back to (height, width) format for plotting
img_np = np.array(img)

# Plot the image
fig, ax = plt.subplots(1)
ax.imshow(img_np)

# Plot each bounding box
for box in original_boxes:
    # The coordinates of the box are in [y_min, x_min, y_max, x_max] format
    y_min, x_min, y_max, x_max = box
    
    # Calculate width and height of the box
    width = x_max - x_min
    height = y_max - y_min
    
    # Create a rectangle patch
    rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
    
    # Add the patch to the Axes
    ax.add_patch(rect)

# Display the plot
plt.show()
